### Información extraida del PDF

In [2]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./data/los7HabitosGenteAltamenteEfectiva.pdf" #Ruta al archivo PDF
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs)) # Imprimimos la cantidad de partes del PDF

206


# Version paga

In [ ]:
from dotenv import load_dotenv
import os

# Cargar variables del archivo .env
load_dotenv()

# Leer la API key desde el entorno
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [4]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

### Ahora la VectorDB es persistente, se guardan los datos en esta carpeta

In [5]:
import chromadb

persistent_client = chromadb.PersistentClient(path='./vectordb')

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200) #el text splitter nos va a permitir partir los documentos con el tamaño de tokens que queramos

splits = text_splitter.split_documents(docs) #partimos el pdf en mas partes

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(), persist_directory='./vectordb') #creamos la base de datos de vectores

retriever = vectorstore.as_retriever() #unimos todo y damos la posibilidad de hacer consultas

### Resultados

In [10]:
retriever.get_relevant_documents('afilar la sierra') #consulta que nos va a traer los documentos mas reelevantes teniendo en cuenta nuestra consulta

/tmp/ipykernel_84313/275184945.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents('afilar la sierra') #consulta que nos va a traer los documentos mas reelevantes teniendo en cuenta nuestra consulta


[Document(id='cc0b002b-4469-4c1f-981c-dc749258d3b8', metadata={'author': 'STEPHEN R. COVEY', 'title': 'Los 7 hábitos de la gente altamente efectiva', 'creator': 'Covey, Stephen R. - Los 7 hábitos de la gente altamente efectiva.doc - Microsoft Word', 'page': 176, 'creationdate': '2005-07-09T12:39:45+00:00', 'moddate': '2013-10-29T10:31:20-03:00', 'page_label': '177', 'producer': 'Acrobat PDFWriter 4.0 for Windows NT', 'source': './los7HabitosGenteAltamenteEfectiva.pdf', 'total_pages': 206}, page_content='George Shee -han se refiere a cuatro roles: ser un buen animal (físico), un buen artesano (mental), un buen \namigo (social) y un santo (espiritual). La teoría seria de la motivación y las organizaciones abarca esas cuatro \ndimensiones de la motivación: la economía (física); el modo en que la gente es tratada (social); el modo en que \nla gente es desarrollada y utilizada (mental); y el servicio, el puesto de trabajo, la contribución de la \norganización (espiritual).  \n«Afilar la sie

### LLM

In [11]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)

### Crear prompt

In [12]:
system_prompt = (
    "Tú eres un asistente para tareas de respuesta a preguntas."
    "Usa los siguientes fragmentos de contexto recuperado para responder "
    "la pregunta. Si no sabes la respuesta, di que no "
    "sabes. Usa un máximo de tres oraciones y mantén la "
    "respuesta concisa."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


chain = create_stuff_documents_chain(llm, prompt)
rag = create_retrieval_chain(retriever, chain)

### Obtener nuestra respuesta

In [13]:
results = rag.invoke({"input": "cual es el septimo habito?"})

In [14]:
results

{'input': 'cual es el septimo habito?',
 'context': [Document(id='a1511de7-c7f3-4d3a-9274-fb0b83de033f', metadata={'page': 176, 'total_pages': 206, 'author': 'STEPHEN R. COVEY', 'page_label': '177', 'producer': 'Acrobat PDFWriter 4.0 for Windows NT', 'creationdate': '2005-07-09T12:39:45+00:00', 'title': 'Los 7 hábitos de la gente altamente efectiva', 'source': './los7HabitosGenteAltamenteEfectiva.pdf', 'creator': 'Covey, Stephen R. - Los 7 hábitos de la gente altamente efectiva.doc - Microsoft Word', 'moddate': '2013-10-29T10:31:20-03:00'}, page_content='aserrando. \nEl séptimo hábito consiste en tomar tiempo para afilar la sierra. Engloba a todos los otros hábitos del \nmodelo de los siete hábitos por que es el que los hace posibles.  \n \n \nLas cuatro dimensiones de la renovación \n \n \nEl séptimo hábito es CP personal. Significa preservar y realzar el mayor bien que usted posee: usted \nmismo. Significa renovar las cua tro dimensiones de su naturaleza: la física, la espiritual, la

In [15]:
results['answer']

'El séptimo hábito es tomar tiempo para afilar la sierra, que implica renovar las cuatro dimensiones de su naturaleza: física, espiritual, mental y social/emocional. Este hábito es crucial ya que permite preservar y realzar el mayor bien que uno posee: uno mismo.'

In [16]:
rag.invoke({"input": "cual es el primer habito"})

{'input': 'cual es el primer habito',
 'context': [Document(id='0b179034-0a83-4f76-a84f-6af6266363e4', metadata={'source': './los7HabitosGenteAltamenteEfectiva.pdf', 'creator': 'Covey, Stephen R. - Los 7 hábitos de la gente altamente efectiva.doc - Microsoft Word', 'creationdate': '2005-07-09T12:39:45+00:00', 'page': 89, 'moddate': '2013-10-29T10:31:20-03:00', 'page_label': '90', 'total_pages': 206, 'title': 'Los 7 hábitos de la gente altamente efectiva', 'author': 'STEPHEN R. COVEY', 'producer': 'Acrobat PDFWriter 4.0 for Windows NT'}, page_content='El tercer hábito es el fruto personal, la realización práctica del primero y el segundo.  \nEl primer hábito dice: «Tú eres el creador. Tú estás a cargo de todo». Se basa en los cuatro privilegios \nhumanos de la imaginació n, la conciencia moral, la voluntad independiente y, en particular, la autoconciencia. \nNos permite decir: «Éste es un programa deficiente que yo recibí en mi infancia procedente de mi espejo social. \nNo me gusta ese 

### Una vez ya lo tenemos guardado en disco

In [17]:
chroma_local = Chroma(persist_directory="./vectordb", embedding_function=OpenAIEmbeddings())

In [18]:
retriever = chroma_local.as_retriever()

In [19]:
retriever.get_relevant_documents('afilar la sierra') #consulta que nos va a traer los documentos mas reelevantes teniendo en cuenta nuestra consulta

[Document(id='cc0b002b-4469-4c1f-981c-dc749258d3b8', metadata={'creationdate': '2005-07-09T12:39:45+00:00', 'page_label': '177', 'title': 'Los 7 hábitos de la gente altamente efectiva', 'author': 'STEPHEN R. COVEY', 'producer': 'Acrobat PDFWriter 4.0 for Windows NT', 'page': 176, 'total_pages': 206, 'creator': 'Covey, Stephen R. - Los 7 hábitos de la gente altamente efectiva.doc - Microsoft Word', 'source': './los7HabitosGenteAltamenteEfectiva.pdf', 'moddate': '2013-10-29T10:31:20-03:00'}, page_content='George Shee -han se refiere a cuatro roles: ser un buen animal (físico), un buen artesano (mental), un buen \namigo (social) y un santo (espiritual). La teoría seria de la motivación y las organizaciones abarca esas cuatro \ndimensiones de la motivación: la economía (física); el modo en que la gente es tratada (social); el modo en que \nla gente es desarrollada y utilizada (mental); y el servicio, el puesto de trabajo, la contribución de la \norganización (espiritual).  \n«Afilar la sie

# Version gratis

In [1]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2') #modelo de embeddings gratis

/tmp/ipykernel_95338/4207004298.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2') #modelo de embeddings gratis


### Ahora la VectorDB es persistente, se guardan los datos en esta carpeta

In [4]:
import chromadb

persistent_client = chromadb.PersistentClient(path='./vectordb_gratis')

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) #usamos menos tokens que en el anterior debido a que este modelo es inferior
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory='./vectordb_gratis')

retriever = vectorstore.as_retriever()

### Resultados

In [6]:
retriever.get_relevant_documents('afilar la sierra') 

/tmp/ipykernel_95338/3081317603.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents('afilar la sierra')


[Document(id='681d5e96-f75c-4cf9-a1e5-63fed78a9220', metadata={'page': 127, 'total_pages': 206, 'creationdate': '2005-07-09T12:39:45+00:00', 'producer': 'Acrobat PDFWriter 4.0 for Windows NT', 'creator': 'Covey, Stephen R. - Los 7 hábitos de la gente altamente efectiva.doc - Microsoft Word', 'moddate': '2013-10-29T10:31:20-03:00', 'title': 'Los 7 hábitos de la gente altamente efectiva', 'author': 'STEPHEN R. COVEY', 'source': './los7HabitosGenteAltamenteEfectiva.pdf', 'page_label': '128'}, page_content='Afile la Sierra  7\nProcure primero\ncomprender…\ny después ser\ncomprendido\n5\nSinergice\n6\nPiense en\nganar/ganar\n4\nVictoria\nPública\nVictoria\nPrivada\n3\nPrimero\nLo primero\n1\nSea \nproactivo\n2\nEmpiece con un \nfin en mente'),
 Document(id='89c91d9a-889c-4337-9b13-9de367ca9836', metadata={'author': 'STEPHEN R. COVEY', 'page_label': '162', 'page': 161, 'total_pages': 206, 'creationdate': '2005-07-09T12:39:45+00:00', 'creator': 'Covey, Stephen R. - Los 7 hábitos de la gente a

### LLM

In [10]:
from langchain_community.chat_models import ChatOllama
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOllama(model='llama3', temperature=0)

### Crear prompt

In [11]:
system_prompt = (
    "Tú eres un asistente para tareas de respuesta a preguntas."
    "Usa los siguientes fragmentos de contexto recuperado para responder "
    "la pregunta. Si no sabes la respuesta, di que no "
    "sabes. Usa un máximo de tres oraciones y mantén la "
    "respuesta concisa."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


chain = create_stuff_documents_chain(llm, prompt)
rag = create_retrieval_chain(retriever, chain)

### Obtener nuestra respuesta

In [12]:
results = rag.invoke({"input": "que es afilar la sierra?"})

In [13]:
results

{'input': 'que es afilar la sierra?',
 'context': [Document(id='a4286492-1657-4860-a851-e9f366f37f26', metadata={'creationdate': '2005-07-09T12:39:45+00:00', 'title': 'Los 7 hábitos de la gente altamente efectiva', 'creator': 'Covey, Stephen R. - Los 7 hábitos de la gente altamente efectiva.doc - Microsoft Word', 'producer': 'Acrobat PDFWriter 4.0 for Windows NT', 'total_pages': 206, 'moddate': '2013-10-29T10:31:20-03:00', 'page': 27, 'source': './los7HabitosGenteAltamenteEfectiva.pdf', 'author': 'STEPHEN R. COVEY', 'page_label': '28'}, page_content='y conoceremos el lugar por primera vez.'),
  Document(id='0a139db6-4da8-4bfc-a307-39b9cf1d1993', metadata={'page_label': '177', 'page': 176, 'producer': 'Acrobat PDFWriter 4.0 for Windows NT', 'creationdate': '2005-07-09T12:39:45+00:00', 'creator': 'Covey, Stephen R. - Los 7 hábitos de la gente altamente efectiva.doc - Microsoft Word', 'moddate': '2013-10-29T10:31:20-03:00', 'total_pages': 206, 'source': './los7HabitosGenteAltamenteEfectiv

In [14]:
results['answer']

'Según el fragmento de contexto, "afilar la sierra" se refiere a tomar un tiempo para descansar o hacer una pausa durante la tarea. En este caso, el hombre está trabajando más de cinco horas y siente que no tiene tiempo para hacerlo, pero el otro personaje sugiere que afilando la sierra (tomando un descanso) podría ayudarlo a trabajar más rápido y eficientemente.'

In [15]:
rag.invoke({"input": "cual es el habito 5?"})

{'input': 'cual es el habito 5?',
 'context': [Document(id='be7aa1e0-c6df-44c6-be57-85417d1cb17a', metadata={'title': 'Los 7 hábitos de la gente altamente efectiva', 'producer': 'Acrobat PDFWriter 4.0 for Windows NT', 'page_label': '32', 'creationdate': '2005-07-09T12:39:45+00:00', 'source': './los7HabitosGenteAltamenteEfectiva.pdf', 'total_pages': 206, 'moddate': '2013-10-29T10:31:20-03:00', 'creator': 'Covey, Stephen R. - Los 7 hábitos de la gente altamente efectiva.doc - Microsoft Word', 'author': 'STEPHEN R. COVEY', 'page': 31}, page_content='hábitos 4, 5 y 6. Comprender la secuencia ayuda a controlar el desarrollo con más efectividad, pero no le \nsugiero al lector que se  aísle durante varios años hasta desarro llar completamente los hábitos 1, 2 y 3.  \nComo parte de un mundo interdependiente, uno tiene que rela cionarse con ese mundo día tras día. Pero \nlos problemas más apre miantes de ese mundo pueden fácilmente oscurecer las causas de carácter crónico. La'),
  Document(id='

### Una vez ya lo tenemos guardado en disco

In [16]:
chroma_local = Chroma(persist_directory="./vectordb_gratis", embedding_function=embeddings)

In [17]:
retriever = chroma_local.as_retriever()

In [18]:
retriever.get_relevant_documents('habito 6')

[Document(id='f57d410d-d3e0-4a64-b1af-1b3213e256b3', metadata={'page': 185, 'source': './los7HabitosGenteAltamenteEfectiva.pdf', 'moddate': '2013-10-29T10:31:20-03:00', 'title': 'Los 7 hábitos de la gente altamente efectiva', 'page_label': '186', 'producer': 'Acrobat PDFWriter 4.0 for Windows NT', 'creationdate': '2005-07-09T12:39:45+00:00', 'creator': 'Covey, Stephen R. - Los 7 hábitos de la gente altamente efectiva.doc - Microsoft Word', 'total_pages': 206, 'author': 'STEPHEN R. COVEY'}, page_content='(segundo hábito) y la admi nistración personal (tercer hábito) en la vida. Cuanto más efectiva mente \nadministramos nuestra vida (tercer hábito), en mayor medida podemos realizar activ idades renovadoras del \ncuadrante II (séptimo hábito). Cuanto más procuramos primero comprender (quinto hábi to), con más efectividad \npodemos buscar soluciones sinérgicas del tipo ganar/ganar (cuarto y sexto hábitos). Cuanto más progresamos'),
 Document(id='b956e63e-ec41-4c40-b52e-10d714544376', metad